In this notebook, we build a simple taxonomy based on the keywords of a specific field. 

In [ ]:
#@title Fll out the form and hit the play button on the top left!
token = ""  #@param {type: "string"}
dataset_id = '' #@param {type: "string"}
model_name = "" #@param {type: "string"}
text_field = "" #@param {type: "string"}
#@markdown Enter the vector field to label against
vector_fields = ["sample_vector_"] #@param
#@markdown Enter the taxonomy to label against
output_field = "_taxonomy_" #@param {type: "string"}

#@markdown Optional Configurations
taxonomy_size = 30 #@param
keyphrase_ngram_range = (1, 2) #@param
stop_words = None #@param

!pip install -q -U RelevanceAI-dev
!pip install -q keybert

from relevanceai import Client 
from keybert import KeyBERT
client = Client(token=token)
ds = client.Dataset(dataset_id)


kw_model = KeyBERT(model=model_name)

def extract_keywords(docs):
    for d in docs:
        keywords = kw_model.extract_keywords(
            d[text_field], 
            keyphrase_ngram_range=keyphrase_ngram_range, 
            stop_words=stop_words
        )
        d['_keyword_'] = [{"keyword": k, "score": s} for k, s in keywords]
    return docs
ds.bulk_apply(extract_keywords)
top_taxonomy = list(set([x['value'] for x in ds.facets(['_keyword_.keyword'], 
    page_size=taxonomy_size)['results']['_keyword_.keyword']]))

print("Suggested Taxonomy: ")
from pprint import pprint
pprint(top_taxonomy)

label_ds = client.Dataset(dataset_id + "_labels")
label_ds.insert_list(top_taxonomy, create_id=True)
label_ds.vectorize_text(
    fields=['label'],
    output_fields=["label_vector_"]
)

ds.label_from_dataset(
    vector_fields=vector_fields,
    label_dataset=label_ds,
    output_field=output_field
)

ds.facets([output_field])